# REPORT 1 - Testing different boosting priority strategies

In this notebook we test different age-priority boosting strategies, using the `warwickmodel`, built by Universities of Warwick and Lancaster, with population data from 5 countries with very different socio-economic profiles. We assume an initial boosting campaign in the population, with no subsequent boosters being deployed during the simulation. 

The infection dynamics are run for:
 - Dates: **15 Feb 2020** - **25 June 2021**;
 - Countries of interest: **United Kingdom**, **France**;
 - Number of boosters deployed: **10%** of the population.

*The Warwick model is built by Universities of Warwick and Lancaster.*

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
import scipy
import epimodels as em
import warwickmodel as wm
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the WarwickLanc Model

In [2]:
# Populate the model
total_days =  100
regions = ['United Kingdom', 'France']
age_groups = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39',
              '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']

regimes = np.arange(1, 101, 1).tolist()

# Add folder path to data file
path = os.path.join('../data/')

### Read in corresponding data files for the countries considered for all 100 regimes

In [3]:
# Matrices contact
matrices_contact = []
matrices_region = []
time_changes_contact = []
time_changes_region = []

# Vaccine effects
nu_tra = []
nu_symp = []
nu_inf = []
nu_sev_h = []
nu_sev_d = []

# Parameters
omega = []
alpha = []
gamma = []
tau = []
we = []

# Initial conditions
susceptibles_IC = []
exposed1_IC = []
exposed2_IC = []
exposed3_IC = []
exposed4_IC = []
exposed5_IC = []
infectives_sym_IC = []
infectives_asym_IC = []
recovered_IC = []

# Risk factors
d = []
beta = []

# Probabilities of proceeding to severe outcomes
pItoH = []
pHtoD = []

# Distribution of delays before proceeding to severe outcomes
dItoH = []
dHtoD = []

for R in regimes:
        regimes_matrices_region = []

        # Initial state of the system
        weeks_matrices_region = []
        for r in regions:
                region_data_matrix = pd.read_csv(
                        os.path.join(path, '{}/Contacts_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)
                regional = em.RegionMatrix(r, age_groups, region_data_matrix)
                weeks_matrices_region.append(regional)

        regimes_matrices_region.append(weeks_matrices_region)

        contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
        regimes_matrices_contact = [contacts]

        matrices_region.append(regimes_matrices_region)
        matrices_contact.append(regimes_matrices_contact)

        # Matrices contact
        time_changes_contact.append([1])
        time_changes_region.append([1])

        # Over 75 population fractions
        frac_pop_over75 = []

        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        skiprows=15,
                        header=None, dtype=np.float64)

                frac_pop_over75.append((1/np.sum(np.asarray(IC_df))) * np.sum(np.asarray(IC_df),axis=1))

        # Risk Factors
        extended_regimes_d = []
        regimes_d = []
        regimes_beta = []

        for r, reg in enumerate(regions):
                RF_df = pd.read_csv(
                        os.path.join(path, '{}/Risks_{}.csv'.format(reg, R)),
                        dtype=np.float64)
                extended_d = RF_df['symptom_risk'].tolist()
                extended_beta = RF_df['susceptibility'].tolist()

                extended_regimes_d.append(extended_d)

                regimes_d.append(extended_d[:15] + [np.sum(np.multiply(extended_d[15:], frac_pop_over75[r]))])
                regimes_beta.append(extended_beta[:15] + [np.sum(np.multiply(extended_beta[15:], frac_pop_over75[r]))])

        d.append(regimes_d)
        beta.append(regimes_beta)

        # Vaccine effects
        eff_df = pd.read_csv(
                os.path.join(path, 'global_parameters/efficacies_{}.csv'.format(R)),
                usecols=range(1,5), dtype=np.float64)

        VE_i = eff_df['Infection_eff']
        VE_s = eff_df['Symptom_eff']
        VE_h = eff_df['Hosp_eff']
        VE_d = eff_df['Death_eff']

        VE_d = np.divide(VE_d-VE_h, 1-VE_h)
        VE_h = np.divide(VE_h-VE_i, 1-VE_i)
        VE_s = np.divide(VE_s-VE_i, 1-VE_i)

        regimes_nu_tra = [1] * 6
        regimes_nu_symp = np.nan_to_num(1 - VE_s).tolist()
        regimes_nu_inf = np.nan_to_num(1 - VE_i).tolist()
        regimes_nu_sev_h = np.nan_to_num(1 - VE_h).tolist()
        regimes_nu_sev_d = np.nan_to_num(1 - VE_d).tolist()

        nu_tra.append(regimes_nu_tra)
        nu_symp.append(regimes_nu_symp)
        nu_inf.append(regimes_nu_inf)
        nu_sev_h.append(regimes_nu_sev_h)
        nu_sev_d.append(regimes_nu_sev_d)

        # Parameters
        param_df = pd.read_csv(
                os.path.join(path, 'global_parameters/parameters_{}.csv'.format(R)),
                dtype=np.float64)

        regimes_omega = param_df['transmission'].tolist()[0]
        regimes_alpha = 1
        regimes_gamma = param_df['recovery'].tolist()[0]
        regimes_tau = param_df['asymptomatic_transmission'].tolist()[0]
        regimes_we = [param_df['waning_rate'].tolist()[0]] * 2 + [0]

        omega.append(regimes_omega)
        alpha.append(regimes_alpha)
        gamma.append(regimes_gamma)
        tau.append(regimes_tau)
        we.append(regimes_we)

        # Initial conditions
        regimes_susceptibles_IC = []
        regimes_exposed1_IC = []
        regimes_exposed2_IC = []
        regimes_exposed3_IC = []
        regimes_exposed4_IC = []
        regimes_exposed5_IC = []
        regimes_infectives_sym_IC = []
        regimes_infectives_asym_IC = []
        regimes_recovered_IC = []

        # Susceptible
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(0, 5),
                        header=None, dtype=np.float64)

                extended_S = np.asarray(IC_df)
                under_75_S = extended_S[:15, :]
                over_75_S = extended_S[15:, :]
                reduced_S = np.vstack((under_75_S, np.sum(over_75_S, axis=0)))
                regimes_susceptibles_IC.append(
                        reduced_S.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 1
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(5, 10),
                        header=None, dtype=np.float64)

                extended_E1 = np.asarray(IC_df)
                under_75_E1 = extended_E1[:15, :]
                over_75_E1 = extended_E1[15:, :]
                reduced_E1 = np.vstack((under_75_E1, np.sum(over_75_E1, axis=0)))
                regimes_exposed1_IC.append(
                        reduced_E1.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 2
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(10, 15),
                        header=None, dtype=np.float64)

                extended_E2 = np.asarray(IC_df)
                under_75_E2 = extended_E2[:15, :]
                over_75_E2 = extended_E2[15:, :]
                reduced_E2 = np.vstack((under_75_E2, np.sum(over_75_E2, axis=0)))
                regimes_exposed2_IC.append(
                        reduced_E2.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 3
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(15, 20),
                        header=None, dtype=np.float64)

                extended_E3 = np.asarray(IC_df)
                under_75_E3 = extended_E3[:15, :]
                over_75_E3 = extended_E3[15:, :]
                reduced_E3 = np.vstack((under_75_E3, np.sum(over_75_E3, axis=0)))
                regimes_exposed3_IC.append(
                        reduced_E3.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 4
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(20, 25),
                        header=None, dtype=np.float64)

                extended_E4 = np.asarray(IC_df)
                under_75_E4 = extended_E4[:15, :]
                over_75_E4 = extended_E4[15:, :]
                reduced_E4 = np.vstack((under_75_E4, np.sum(over_75_E4, axis=0)))
                regimes_exposed4_IC.append(
                        reduced_E4.flatten('F').tolist() + [0] * len(age_groups))

        # Exposed 5
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(25, 30),
                        header=None, dtype=np.float64)

                extended_E5 = np.asarray(IC_df)
                under_75_E5 = extended_E5[:15, :]
                over_75_E5 = extended_E5[15:, :]
                reduced_E5 = np.vstack((under_75_E5, np.sum(over_75_E5, axis=0)))
                regimes_exposed5_IC.append(
                        reduced_E5.flatten('F').tolist() + [0] * len(age_groups))

        # Symptomatic & Asymptomatic Infectious
        for _, r in enumerate(regions):
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=range(30, 35),
                        header=None, dtype=np.float64)

                extended_I = np.zeros_like(np.asarray(IC_df))
                extended_I[:, 0] = np.matmul(np.diag(regimes_nu_symp[0] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 0])
                extended_I[:, 1] = np.matmul(np.diag(regimes_nu_symp[1] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 1])
                extended_I[:, 2] = np.matmul(np.diag(regimes_nu_symp[2] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 2])
                extended_I[:, 3] = np.matmul(np.diag(regimes_nu_symp[3] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 3])
                extended_I[:, 4] = np.matmul(np.diag(regimes_nu_symp[4] * np.array(extended_regimes_d[_])), np.asarray(IC_df)[:, 4])
                under_75_I = extended_I[:15, :]
                over_75_I = extended_I[15:, :]
                reduced_I = np.vstack((under_75_I, np.sum(over_75_I, axis=0)))
                regimes_infectives_sym_IC.append(
                        reduced_I.flatten('F').tolist() + [0] * len(age_groups))

                extended_A = np.zeros_like(np.asarray(IC_df))
                extended_A[:, 0] = np.matmul(np.diag((1 - regimes_nu_symp[0] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 0])
                extended_A[:, 1] = np.matmul(np.diag((1 - regimes_nu_symp[1] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 1])
                extended_A[:, 2] = np.matmul(np.diag((1 - regimes_nu_symp[2] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 2])
                extended_A[:, 3] = np.matmul(np.diag((1 - regimes_nu_symp[3] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 3])
                extended_A[:, 4] = np.matmul(np.diag((1 - regimes_nu_symp[4] * np.array(extended_regimes_d[_]))), np.asarray(IC_df)[:, 4])
                under_75_A = extended_A[:15, :]
                over_75_A = extended_A[15:, :]
                reduced_A = np.vstack((under_75_A, np.sum(over_75_A, axis=0)))
                regimes_infectives_asym_IC.append(
                        reduced_A.flatten('F').tolist() + [0] * len(age_groups))

        # Recovered
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        usecols=[35],
                        header=None, dtype=np.float64)

                extended_R = np.asarray(IC_df)
                under_75_R = extended_R[:15, :]
                over_75_R = extended_R[15:, :]
                reduced_R = np.vstack((under_75_R, np.sum(over_75_R, axis=0)))
                regimes_recovered_IC.append(
                        reduced_R.flatten('F').tolist())

        susceptibles_IC.append(regimes_susceptibles_IC)
        exposed1_IC.append(regimes_exposed1_IC)
        exposed2_IC.append(regimes_exposed2_IC)
        exposed3_IC.append(regimes_exposed3_IC)
        exposed4_IC.append(regimes_exposed4_IC)
        exposed5_IC.append(regimes_exposed5_IC)
        infectives_sym_IC.append(regimes_infectives_sym_IC)
        infectives_asym_IC.append(regimes_infectives_asym_IC)
        recovered_IC.append(regimes_recovered_IC)

        # Set time-to-hospitalisation using a Gamma distribution using the mean and standard deviation 
        th_mean = param_df['hosp_lag'].tolist()[0]+0.00001
        th_var = 12.1**2
        theta = th_var / th_mean
        k = th_mean / theta
        time_to_hosp = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Set time-to-death using a Gamma distribution using the mean and standard deviation
        td_mean = param_df['death_lag'].tolist()[0]
        td_var = 12.1**2
        theta = td_var / td_mean
        k = td_mean / theta
        time_to_death = scipy.stats.gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

        # Probabilities of proceeding to severe outcomes
        # Infected -> Hospital
        extended_pItoH = RF_df['hospitalisation_risk'].tolist()

        regimes_pItoH = []
        for r, reg in enumerate(regions):
                regimes_pItoH.append(extended_pItoH[:15] + [np.sum(np.multiply(extended_pItoH[15:], frac_pop_over75[r]))])

        pItoH.append(regimes_pItoH)

        # Hospital -> Death
        extended_pHtoD = RF_df['death_risk'].tolist()

        regimes_pHtoD = []
        for r, reg in enumerate(regions):
                regimes_pHtoD.append(extended_pHtoD[:15] + [np.sum(np.multiply(extended_pHtoD[15:], frac_pop_over75[r]))])

        pHtoD.append(regimes_pHtoD)

        # Distribution of delays before proceeding to severe outcomes
        # Infected -> Hospital
        dItoH.append(time_to_hosp)
        # Hospital -> Death
        dHtoD.append(time_to_death)

# Other parameters
vac=0
vacb=0

adult = np.ones(len(age_groups))
adult[0] = 0
adult[1] = 0
adult[2] = 0

### Calculate maximum number of booster vaccines available

In [4]:
# Proportion of the total population for which boosters are available
boost_pop_percent = 0.1

# Compute the number of boosters for each region
boosters = []

for R in regimes:
        regimes_boosters = []
        for r in regions:
                IC_df = pd.read_csv(
                        os.path.join(path, '{}/Start_pop_{}.csv'.format(r, R)),
                        header=None, dtype=np.float64)

                regimes_boosters.append(boost_pop_percent * np.sum(np.asarray(IC_df)))
        boosters.append(regimes_boosters)

## Boosting campaign scenarios

We only boost the partially-waned and recovered. However those in the R compartment who receive the booster do not move out of the compartment (they have higher immunity than the boosted)

In [5]:
# Maximum percentage booster uptake of each age group
boost_age_percent = 0.9

# Compute the maximum number of boosters we can deploy for each age group in each region
max_boosters = []
max_boosters_for_R = []
max_boosters_for_Sw1 = []

for R, _ in enumerate(regimes):
        regimes_max_boosters = []
        regimes_max_boosters_for_R = []
        regimes_max_boosters_for_Sw1 = []
        for r, reg in enumerate(regions):
                boosters_for_R = boost_age_percent * np.asarray(recovered_IC[R][r])
                boosters_for_Sw1 = boost_age_percent * np.asarray(susceptibles_IC[R][r])[(3*len(age_groups)):(4*len(age_groups))]

                regimes_max_boosters_for_R.append(boosters_for_R)
                regimes_max_boosters_for_Sw1.append(boosters_for_Sw1)
                regimes_max_boosters.append(boosters_for_R + boosters_for_Sw1)

        max_boosters_for_R.append(regimes_max_boosters_for_R)
        max_boosters_for_Sw1.append(regimes_max_boosters_for_Sw1)
        max_boosters.append(regimes_max_boosters)

# Create list of new susceptible_ICs for each vaccination scenario
scenario_susceptibles_IC = []
scenario_names = []
age_boosting_scenario_order = []

**Scenario 1**: Prioritising those 75+

In [6]:
scenario_names.append('Prioritise 75+')
age_boosting_scenario_order.append((np.arange(16, 0, -1) - 1).tolist())

**Scenario 2**: Prioritising those 60-70 then 75+

In [7]:
scenario_names.append('Prioritise 60-70 then 75+')
age_boosting_scenario_order.append([range(13, 15), 15] + (np.arange(13, 0, -1) - 1).tolist())

**Scenario 3**: Prioritising those 60-70 then 50-60, then 75+

In [8]:
scenario_names.append('Prioritise those 60-70 then 50-60, then 75+')
age_boosting_scenario_order.append([range(13, 15), range(11, 13), 15] + (np.arange(11, 0, -1) - 1).tolist())

**Scenario 4**: Prioritising those 50-60, then 60-70 then 75+

In [9]:
scenario_names.append('Prioritise those 50-60 then 60-70, then 75+')
age_boosting_scenario_order.append([range(11, 13), range(13, 15), 15] + (np.arange(11, 0, -1) - 1).tolist())

**Scenario 5**: Prioritising those 50-69, then 60-70 then 75+

### Compute new susceptible conditions based on each boosting scenario

In [10]:
for s, _ in enumerate(scenario_names):
    regimes_new_susceptibles_IC = []
    scenario_boosters = boosters.copy()

    for R, _ in enumerate(regimes):
        new_susceptibles_IC = [
            np.array(susceptibles_IC[R][r].copy()).reshape((6, 16)).transpose() for r in range(len(regions))]
        
        new_susceptibles_IC = np.array(new_susceptibles_IC)
        reg_new_susceptibles_IC = []

        for r, reg in enumerate(regions):
            for ages in age_boosting_scenario_order[s]:
                if np.sum(max_boosters[R][r][ages]) <= scenario_boosters[R][r]:
                    # Add boosted from the Sw1 to Sb
                    new_susceptibles_IC[r, ages, 2] += max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw1
                    new_susceptibles_IC[r, ages, 3] -= max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw1 and R from total boocters for the scenario
                    scenario_boosters[R][r] -= np.sum(max_boosters[R][r][ages])
                else:
                    # Compute proportion of boosters we have left to give for the age group
                    prop = scenario_boosters[R][r] / np.sum(max_boosters[R][r][ages])
                    # Add boosted from the Sw1 to Sb
                    new_susceptibles_IC[r, ages, 2] += prop * max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw1
                    new_susceptibles_IC[r, ages, 3] -= prop * max_boosters_for_Sw1[R][r][ages]
                    # Remove boosted from the Sw1 and R from total boocters for the scenario
                    scenario_boosters[R][r] -= prop * np.sum(max_boosters[R][r][ages])

            reg_new_susceptibles_IC.append(list(deepflatten(new_susceptibles_IC[r].transpose())))

        regimes_new_susceptibles_IC.append(reg_new_susceptibles_IC)
    scenario_susceptibles_IC.append(regimes_new_susceptibles_IC)

### Set the parameters and initial conditions of the model and bundle everything together

In [11]:
# Instantiate model
model = wm.WarwickLancSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

### Simulate for the regions

In [12]:
# Simulate for all the regions and regimes
outputs = []

for s, scenario in enumerate(scenario_names):
    scenario_outputs = []
    for R, regime in enumerate(regimes):
        model.read_contact_data(matrices_contact[R], time_changes_contact[R])
        model.read_regional_data(matrices_region[R], time_changes_region[R])

        # Set regional and time dependent parameters
        regional_parameters = wm.RegParameters(
            model=model,
            region_index=1
        )

        # Set ICs parameters
        ICs_parameters = wm.ICs(
            model=model,
            susceptibles_IC=scenario_susceptibles_IC[s][R],
            exposed1_IC=exposed1_IC[R],
            exposed2_IC=exposed2_IC[R],
            exposed3_IC=exposed3_IC[R],
            exposed4_IC=exposed4_IC[R],
            exposed5_IC=exposed5_IC[R],
            infectives_sym_IC=infectives_sym_IC[R],
            infectives_asym_IC=infectives_asym_IC[R],
            recovered_IC=recovered_IC[R]
        )

        # Set disease-specific parameters
        disease_parameters = wm.DiseaseParameters(
            model=model,
            d=d[R][0],
            tau=tau[R],
            we=we[R],
            omega=omega[R]
        )

        # Set transmission parameters
        transmission_parameters = wm.Transmission(
            model=model,
            beta=beta[R][0],
            alpha=alpha[R],
            gamma=gamma[R]
        )

        # Set other simulation parameters
        simulation_parameters = wm.SimParameters(
            model=model,
            method='Radau',
            times=times,
            eps=False
        )

        # Set vaccination parameters
        vaccine_parameters = wm.VaccineParameters(
            model=model,
            vac=vac,
            vacb=vacb,
            adult=adult,
            nu_tra=nu_tra[R],
            nu_symp=nu_symp[R],
            nu_inf=nu_inf[R],
            nu_sev_h=nu_sev_h[R],
            nu_sev_d=nu_sev_d[R],
        )

        # Set social distancing parameters
        soc_dist_parameters = wm.SocDistParameters(
            model=model,
            phi=1
        )

        # Set all parameters in the controller
        parameters = wm.ParametersController(
            model=model,
            regional_parameters=regional_parameters,
            ICs_parameters=ICs_parameters,
            disease_parameters=disease_parameters,
            transmission_parameters=transmission_parameters,
            simulation_parameters=simulation_parameters,
            vaccine_parameters=vaccine_parameters,
            soc_dist_parameters=soc_dist_parameters
        )

        # Simulate for all the regions
        regimes_outputs = []
        for r, reg in enumerate(regions):
            # List of initial conditions and parameters that characterise the model
            parameters.regional_parameters.region_index = r + 1

            parameters.disease_parameters.d = d[R][r]
            parameters.transmission_parameters.beta = beta[R][r]

            # Simulate using the ODE solver
            regimes_outputs.append(model.simulate(parameters))

        scenario_outputs.append(regimes_outputs)

    outputs.append(scenario_outputs)

outputs = np.array(outputs)

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [13]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']

## Number of New Infections, Hospitalisations & Deaths

In [14]:
# Simulate for all the regions
total_new_deaths = []

for s, scenario in enumerate(scenario_names):
    scenario_total_new_deaths = []
    for R, regime in enumerate(regimes):
        regimes_total_new_deaths = []
        for r, reg in enumerate(regions):
            # Compute regional matrix of new infections for all timepoints simulated
            reg_new_infections = model.new_infections(outputs[s, R, r, :, :])

            # Compute regional matrix of new hospitalisation for all timepoints simulated
            reg_new_hospitalisation = model.new_hospitalisations(reg_new_infections, pItoH[R][r], dItoH[R])

            # Compute regional matrix of new deaths for all timepoints simulated
            reg_new_deaths = model.new_deaths(reg_new_hospitalisation, pHtoD[R][r], dHtoD[R])

            regimes_total_new_deaths.append(np.sum(
                reg_new_deaths[0] + reg_new_deaths[1] + reg_new_deaths[2] +
                reg_new_deaths[3] + reg_new_deaths[4] + reg_new_deaths[5],
                axis=1))

        scenario_total_new_deaths.append(regimes_total_new_deaths)

    total_new_deaths.append(scenario_total_new_deaths)

total_new_deaths = np.array(total_new_deaths)


In [15]:
# Set up traces to plot
total_new_deaths_mean = []
total_new_deaths_upper = []
total_new_deaths_lower = []

for r, _ in enumerate(regions):
    # Compute the mean 
    total_new_deaths_mean.append(np.mean(total_new_deaths[:,:,r,:], axis=1))

    # Compute the upper quantiles
    total_new_deaths_upper.append(np.quantile(total_new_deaths[:,:,r,:], 0.975, axis=1))

    # Compute the lower qunatiles
    total_new_deaths_lower.append(np.quantile(total_new_deaths[:,:,r,:], 0.025, axis=1))

### Plot the time series of numbers of deaths of the two regions

In [16]:
# Trace names - represent the solver used for the simulation
trace_name = ['region {}'.format(r) for r in regions]

# Plot for each boosting scenario
for s, scenario in enumerate(scenario_names):
    fig = go.Figure()
    # Plot (line plot for each solver method for each age)
    for r, reg in enumerate(regions):
        fig.add_trace(
            go.Scatter(
                y=total_new_deaths_mean[r][s, :],
                x=parameters.simulation_parameters.times,
                mode='lines',
                name=trace_name[r],
                line_color=colours[r],
            )
        )

        fig.add_trace(
            go.Scatter(
                y=total_new_deaths_upper[r][s, :].tolist() + total_new_deaths_lower[r][s, :].tolist()[::-1],
                x=parameters.simulation_parameters.times + parameters.simulation_parameters.times[::-1],
                mode='lines',
                name=trace_name[r],
                fill='toself',
                fillcolor=colours[r],
                line_color=colours[r],
                opacity=0.15,
            )
        )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title='Deaths for Scenario: {}'.format(scenario),
        width=800,
        plot_bgcolor='white',
        xaxis=dict(linecolor='black'),
        yaxis=dict(linecolor='black'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))

    fig.write_image('images/Deaths Scenario {}.pdf'.format(s+1))
    fig.show()